In [3]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from read_bvh import Bone_Tree
def part1_calculate_T_pose(bvh_file_path):
    """请填写以下内容
    输入： bvh 文件路径
    输出:
        joint_name: List[str]，字符串列表，包含着所有关节的名字
        joint_parent: List[int]，整数列表，包含着所有关节的父关节的索引,根节点的父关节索引为-1
        joint_offset: np.ndarray，形状为(M, 3)的numpy数组，包含着所有关节的偏移量

    Tips:
        joint_name顺序应该和bvh一致
    """
    Tree = Bone_Tree(bvh_file_path)
    joint_name = Tree.joint_name
    joint_parent = Tree.joint_parent
    joint_offset = Tree.joint_offset
    return joint_name, joint_parent, joint_offset

In [ ]:
import os

current_directory = os.getcwd()
print(current_directory)
T_pos_bvh_file = os.path.join(current_directory, 'data', 'walk60.bvh')
A_pos_bvh_file = os.path.join(current_directory, 'data', 'A_pos_run.bvh')

T_joint_name, _, _, = part1_calculate_T_pose(T_pos_bvh_file)
A_joint_name, _, _, = part1_calculate_T_pose(A_pos_bvh_file)


In [10]:
T_pos_bvh_file ='walk60.bvh'
A_pos_bvh_file ='A_pos_run.bvh'
T_tree = Bone_Tree('walk60.bvh')
A_tree = Bone_Tree(bvh_file_path= 'A_pose_run.bvh')

In [12]:
T_joint_name = T_tree.joint_name
A_joint_name = A_tree.joint_name


In [13]:
A_motion_data = A_tree.frames

In [14]:
A_joint_map = {}
count = 0
for i in range(len(A_joint_name)):
    if '_end' in A_joint_name[i]:
        count += 1
    A_joint_map[A_joint_name[i]] = i - count

In [15]:
A_joint_map

{'RootJoint': 0,
 'lHip': 1,
 'lKnee': 2,
 'lAnkle': 3,
 'lToeJoint': 4,
 'lToeJoint_end': 4,
 'rHip': 5,
 'rKnee': 6,
 'rAnkle': 7,
 'rToeJoint': 8,
 'rToeJoint_end': 8,
 'pelvis_lowerback': 9,
 'lowerback_torso': 10,
 'torso_head': 11,
 'torso_head_end': 11,
 'lTorso_Clavicle': 12,
 'lShoulder': 13,
 'lElbow': 14,
 'lWrist': 15,
 'lWrist_end': 15,
 'rTorso_Clavicle': 16,
 'rShoulder': 17,
 'rElbow': 18,
 'rWrist': 19,
 'rWrist_end': 19}

In [16]:
motion_data = []

In [19]:
np.array(A_motion_data).shape[0]

14269

In [20]:
A_motion_data = np.array(A_motion_data)

In [21]:
for i in range(A_motion_data.shape[0]):
    data = []
    for joint in T_joint_name:
        index = A_joint_map[joint]
        if joint == 'RootJoint':
            data += list(A_motion_data[i][0:6])
        elif joint == 'lShoulder':
            Rot = (R.from_euler('XYZ', list(A_motion_data[i][index * 3 + 3: index*3 + 6]), degrees=True) * R.from_euler('XYZ', [0., 0., -45.], degrees=True)).as_euler('XYZ',True)
            data += list(Rot)
        elif joint == 'rShoulder':
            Rot = (R.from_euler('XYZ', list(A_motion_data[i][index * 3 + 3: index*3 + 6]), degrees=True) * R.from_euler('XYZ', [0., 0., 45.], degrees=True)).as_euler('XYZ',True)
            data += list(Rot)
        elif '_end' in joint:
            continue
        else:
            data += list(A_motion_data[i][index * 3 + 3: index * 3 + 6])
    motion_data.append(np.array(data).reshape(1, -1))


In [26]:
np.array(motion_data).squeeze().shape

(14269, 63)